In [0]:
!ls

data  datalab  dogscats.zip


In [0]:
!ls

data  datalab  dogscats.zip


In [0]:


%reload_ext autoreload
%autoreload 2
%matplotlib inline



In [0]:


PATH = "data/dogscats/"
sz=224 #image input size
batch_size=64



In [0]:


import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input



In [0]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [0]:


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')



Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
base_model=ResNet50(weights='imagenet',include_top=False)

94658560/94653016 [==============================] - 8s 0us/step


In [0]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(1,activation='sigmoid')(x)

In [0]:
model=Model(inputs=base_model.input,outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 346s 965ms/step - loss: 0.0702 - acc: 0.9773 - val_loss: 0.0656 - val_acc: 0.9859
Epoch 2/3
297/359 [=======================>......] - ETA: 55s - loss: 0.0565 - acc: 0.9814

359/359 [==============================] - 345s 960ms/step - loss: 0.0569 - acc: 0.9816 - val_loss: 0.0989 - val_acc: 0.9773
Epoch 3/3
359/359 [==============================] - 341s 950ms/step - loss: 0.0510 - acc: 0.9836 - val_loss: 0.0651 - val_acc: 0.9874
CPU times: user 31min 44s, sys: 1min 16s, total: 33min
Wall time: 17min 12s
